# Text mining on relevant pages

####  Libraries used

In [154]:
import httplib
import urllib2
import random
import re
import requests
from httplib import IncompleteRead
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sys  
#reload(sys)  
#sys.setdefaultencoding('latin-1')

In [155]:
departement=['Bakel','Bambey','Bignona','Birkelane','Bounkiling','Dagana','Dakar','Diourbel','Fatick','Foundiougne',
            'Gossas','Goudiry','Goudomp','Guediawaye','Guinguineo','Kaffrine','Kanel','Kaolack',
             'Kebemer','Kedougou','Kolda','Koumpentoum','Koungheul','Linguere','Louga','Mbour',
             'Malem Hodar','Matam','Mbacke','Medina Yoro Foulah','Nioro du Rip','Oussouye',
             'Pikine','Podor','Ranerou-Ferlo','Rufisque','Saint-Louis','Salemata','Saraya',
             'Sedhiou','Tambacounda','Thies','Tivaouane','Velingara','Ziguinchor']

arrondissement = ['Almadies','Agnam Civol','Baba Garage','Bandafassi','Brassou','Bala','Bele','Boghal','Bona',
                  'Boyngel Bamba','Bamba Thialene','Barkedji','Bembou','Cas-Cas','Colobane','Coki',
                  'Darou Minam','Dar Salam','Dakateli','Dagoudane','Dodji','Dakar Plateau',
                  'Diakhao','Djilor','Djibanar','Djoulacolon','Diaroume','Darou Mousty','Dianke Makha','Diende',
                  'Djibabouya','Djiredji','Fafacourou','Fissel','Fongolembi', 'Fimela',
                  'Grand Dakar','Gamadji Sare','Guediawaye','Gniby',
                  'Kataba','Kael','Kabrousse','Katakel','Karatanba','Koussanar','Keur Moussa',
                  'Keniaba','Koulor','Koumbal','Kouthiaba Wolof','Keur Momar Sarr','Keur Mboucki',
                  'Lambaye','Lour Escale','Loudia Ouoloff','Ida Mouride',
                  'Mampatim','Makacolibantang','Mouderi','Mbane','Mbadakhoune','Missira Wadine','Missirah','Mbediene',
                  'Ndindy','Ndoulo','Ndiedieng','Ndande','Ndame','Niodor','Ngoye','Ndorna',
                  'Niaming','Niayes','Ndiaye','Niakhar','Notto','Mabo','Medina Sabakh',
                  'Ogo','Orkadiere','Ouadiour', 'Parcelles Assainies', 'Paoskoto','Rao','Rufisque',
                  'Sagatta','Sagatta Dioloff','Sare Bidji','Sindian','Sangalkam','Sagna','Sakal',
                  'Sabodala','Sibassor','Sindia','Sessene','Salde','Thienaba','Thies',
                  'Taif','Tendouck','Tenghory','Tattaguine','Thiaroye','Toubacouta','Thille Boubacar',
                  'Velingara','Wouro Sidy','Wack Ngouna','Yang Yang']


In [156]:
df= pd.read_csv("potential_flood_events.csv",sep=";",encoding="utf-8")

In [157]:
df.head()

,Unnamed: 0,link
0,0,http://actusen.com/index.php/2016/10/17/anelka...
1,1,http://del.icio.us/post?url=http://www.sunuker...
2,2,http://digg.com/submit?phase=2&url=http://www....
3,3,http://laviesenegalaise.com/indemnisation-sini...
4,4,http://laviesenegalaise.com/indemnisation-sini...


## Get the story behind links

In [158]:
df['story']=''
for i , url in enumerate(df.link):
    print i, url
    try:
        req = urllib2.Request(url.decode('utf-8'), headers={'User-Agent' : "Magic Browser"})
        page = urllib2.urlopen( req )
    except urllib2.HTTPError,e:
        continue
    except urllib2.URLError, e:
        continue
    except httplib.HTTPException, e:
        continue
    except Exception:
        continue
    if len(page.read())==0:
        print "the above page does not exist"
        continue
    else:
        soup = BeautifulSoup(page)
        #find all paragraphs from the current "page"
        all_para=[]
        components= soup.find_all('time') + soup.find_all('p')
        for para in components:
            all_para.append(para.text)
        df.loc[i,'story']= ' '.join(all_para).strip()


0 http://actusen.com/index.php/2016/10/17/anelka-repond-a-hollande-il-faut-changer-tout-le-systeme-francais-a-propos-des-enfants-de-la-banlieue/
1 http://del.icio.us/post?url=http://www.sunuker.com/2015/05/26/video-quand-le-parapluie-du-gentleman-obama-embrase-les-reseaux-sociaux/&title=VIDEO – Quand le parapluie du « gentleman » Obama embrase les réseaux sociaux
2 http://digg.com/submit?phase=2&url=http://www.sunuker.com/2015/05/26/video-quand-le-parapluie-du-gentleman-obama-embrase-les-reseaux-sociaux/&title=VIDEO – Quand le parapluie du « gentleman » Obama embrase les réseaux sociaux
3 http://laviesenegalaise.com/indemnisation-sinistres-cices-vers-debut-de-solution
4 http://laviesenegalaise.com/indemnisation-sinistres-cices-vers-debut-de-solution#respond
5 http://laviesenegalaise.com/inondations-saint-louis-eaux-video
6 http://laviesenegalaise.com/inondations-saint-louis-eaux-video#respond
7 http://ledakarois.netbethio-radie-ses-cheikhs-apres-le-ndigueul-du-khalife.html
8 http://sph

### Filter relevancy:

In [159]:
#main subject on the link +1
df['relevancy1']= [1 if re.search("(inondation|inondes)",x) is not None else 0 for x in df.link  ]
# main subject on the content story +1
df['relevancy2']= [1 if re.search("(inondation|inondes|sinistre)",x) is not None else 0 for x in df.story]
# mention of any department on content +1:
df['relevancy3']=[ 1 if x in departement else 0 for x in df.story ]
# mention of any arrondissement on content +1:
df['relevancy4']=[ 1 if x in arrondissement else 0 for x in df.story  ]
df['relevancy']=df['relevancy1']+df['relevancy2']+df['relevancy3']+df['relevancy4']

## Get place by departement

In [168]:
df['place']=''
for i, j in enumerate(df['link']):
    x= [ k for k in departement if k.lower() in j  ]
    if x!=[]:
        if len(x)==1:
            df.loc[i,'place']=str(x)
        if len(x)>1:
            df.loc[i,'place']="-".join(x)
    else:
        df.loc[i,'place']=''
df.place.unique()

array(['', "['Saint-Louis']", "['Dakar']", "['Kolda']", 'Matam-Podor',
       "['Ziguinchor']", "['Kaffrine']"], dtype=object)

## Search place in story content

In [177]:
for i, j in enumerate(df['story']):
    x= [ k for k in departement if k.lower() in j and df.loc[i,'place']==''  ]
    if x!=[]:
        if len(x)==1:
            df.loc[i,'place']=str(x)
        if len(x)>1:
            df.loc[i,'place']="-".join(x)
df.place.unique()

array(['', "['Saint-Louis']", "['Dakar']", "['Kolda']", 'Matam-Podor',
       "['Ziguinchor']", "['Kaffrine']"], dtype=object)

## Get date

In [178]:
df[df.relevancy>0]

,Unnamed: 0,link,story,relevancy1,relevancy2,relevancy3,relevancy4,relevancy,place
5,5,http://laviesenegalaise.com/inondations-saint-...,,1,0,0,0,1,['Saint-Louis']
6,6,http://laviesenegalaise.com/inondations-saint-...,,1,0,0,0,1,['Saint-Louis']
113,113,http://www.senegaltribune.com/articles/1682/1/...,,1,0,0,0,1,
120,120,http://www.senegaltribune.com/articles/2145/1/...,,1,0,0,0,1,
121,121,http://www.senegaltribune.com/articles/2191/1/...,,1,0,0,0,1,
125,125,http://www.senegaltribune.com/articles/2739/1/...,,1,0,0,0,1,
129,129,http://www.senegaltribune.com/articles/2955/1/...,,1,0,0,0,1,
143,143,http://www.senegaltribune.com/articles/3202/1/...,,1,0,0,0,1,
144,144,http://www.senegaltribune.com/articles/3216/1/...,,1,0,0,0,1,
145,145,http://www.senegaltribune.com/articles/3263/1/...,,1,0,0,0,1,
